# Estimating Limits on SHmax

This notebook demonstrates how to estimate limits on the maximum horizontal stress (SHmax) using GeoSuite.

## Overview

This notebook will show you how to:

1. Use wellbore failure data to constrain SHmax
2. Apply stress polygon analysis
3. Estimate upper and lower bounds on SHmax

In [ ]:
# Import GeoSuite modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geosuite.data import load_demo_well_logs
from geosuite.geomech import (
    calculate_overburden_stress,
    calculate_hydrostatic_pressure,
    stress_polygon_limits,
    plot_stress_polygon,
    determine_stress_regime
)

print("GeoSuite imported successfully!")

In [ ]:
# Load demo well log data
df = load_demo_well_logs()

# Find depth and density columns
depth_col = 'depth_m' if 'depth_m' in df.columns else 'DEPTH'
density_cols = [col for col in df.columns if 'RHOB' in col.upper() or 'DENSITY' in col.upper()]

depth = df[depth_col].values
rhob = df[density_cols[0]].values if density_cols else np.ones(len(depth)) * 2.5

# Calculate overburden stress and pore pressure
sv = calculate_overburden_stress(depth, rhob)
pp = calculate_hydrostatic_pressure(depth, rho_water=1.03)

# Estimate Shmin (in practice, use LOT data)
shmin = 0.7 * sv  # Simplified estimate for normal faulting

print(f"Calculated stress profile for {len(depth)} depths")
print(f"Depth range: {depth.min():.1f} to {depth.max():.1f} m")
print(f"Sv range: {sv.min():.1f} to {sv.max():.1f} MPa")

In [ ]:
# Calculate SHmax limits at a specific depth using stress polygon
analysis_depth = 2000.0
idx = np.argmin(np.abs(depth - analysis_depth))

sv_val = sv[idx]
pp_val = pp[idx]
shmin_val = shmin[idx]

limits = stress_polygon_limits(
    sv=sv_val,
    pp=pp_val,
    shmin=shmin_val,
    mu=0.6,
    cohesion=0.0
)

print(f"SHmax Limits at {analysis_depth:.0f} m depth:")
print(f"  Normal faulting: {limits['normal'][0]:.1f} - {limits['normal'][1]:.1f} MPa")
print(f"  Strike-slip: {limits['strike_slip'][0]:.1f} - {limits['strike_slip'][1]:.1f} MPa")
if limits['reverse'][1] is not None:
    print(f"  Reverse faulting: {limits['reverse'][0]:.1f} - {limits['reverse'][1]:.1f} MPa")

print(f"\nMost likely regime: Normal faulting (Shmin < Sv)")
print(f"SHmax likely range: {limits['normal'][0]:.1f} - {limits['normal'][1]:.1f} MPa")

In [ ]:
# Visualize stress polygon
fig = plot_stress_polygon(
    depths=depth,
    sv=sv,
    pp=pp,
    shmin=shmin,
    mu=0.6,
    cohesion=0.0,
    title='Stress Polygon - SHmax Limits'
)

plt.show()